In [1]:
from pathlib import Path

In [ ]:
repo_path = Path("/content/AudioSep")
if not repo_path.exists():
    !git clone https://github.com/Audio-AGI/AudioSep.git

%cd /content/AudioSep

In [2]:
!pip install torchlibrosa==0.1.0 gradio==3.47.1 gdown lightning transformers==4.28.1 ftfy braceexpand webdataset soundfile wget h5py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 66.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.9/308.9 kB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 55.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
checkpoints_dir = Path("checkpoint")
checkpoints_dir.mkdir(exist_ok=True)

models = (
    (
        "https://huggingface.co/spaces/badayvedat/AudioSep/resolve/main/checkpoint/audiosep_base_4M_steps.ckpt",
        checkpoints_dir / "audiosep_base_4M_steps.ckpt"
    ),
    (
        "https://huggingface.co/spaces/badayvedat/AudioSep/resolve/main/checkpoint/music_speech_audioset_epoch_15_esc_89.98.pt",
        checkpoints_dir / "music_speech_audioset_epoch_15_esc_89.98.pt"
    )
)

for model_url, model_path in models:
    if not model_path.exists():
        !wget {model_url} -O {model_path}

In [3]:
!wget "https://audio-agi.github.io/Separate-Anything-You-Describe/demos/exp31_water drops_mixture.wav"

--2025-02-25 12:30:28--  https://audio-agi.github.io/Separate-Anything-You-Describe/demos/exp31_water%20drops_mixture.wav
Resolving audio-agi.github.io (audio-agi.github.io)... 185.199.111.153, 185.199.108.153, 185.199.110.153, ...
Connecting to audio-agi.github.io (audio-agi.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 320044 (313K) [audio/wav]
Saving to: ‘exp31_water drops_mixture.wav’

exp31_water drops_m 100%[===================>] 312.54K  --.-KB/s    in 0.007s  

2025-02-25 12:30:29 (44.9 MB/s) - ‘exp31_water drops_mixture.wav’ saved [320044/320044]



In [3]:
import torch
from pipeline import build_audiosep, separate_audio

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = build_audiosep(
      config_yaml='config/audiosep_base.yaml',
      checkpoint_path=str(models[0][1]),
      device=device)

audio_file = 'exp31_water drops_mixture.wav'
text = 'water drops'
output_file='separated_audio.wav'

# AudioSep processes the audio at 32 kHz sampling rate
separate_audio(model, audio_file, text, output_file, device)

/home/wonjun/miniconda3/envs/AudioSep/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wonjun/miniconda3/envs/AudioSep/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552843/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on an

Loaded AudioSep model from [checkpoint/audiosep_base_4M_steps.ckpt]
Separating audio from [exp31_water drops_mixture.wav] with textual query: [water drops]
Separated audio written to [separated_audio.wav]


In [ ]:
print(f"The separated audio is saved to: '{output_file}' file.")